# CNN Model Creation Version 1:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Loading all the dependencies
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
import random
import pickle
import tensorflow as tf
import tensorflow.keras as keras

In [3]:
# Unpickling the datasets
with open('/content/drive/MyDrive/Projects/Machine Learning Projects/Drowsiness Detection Project/Pickle_Files/img_pck.pickle','rb') as images:
    img_arr=pickle.load(images)

with open('/content/drive/MyDrive/Projects/Machine Learning Projects/Drowsiness Detection Project/Pickle_Files/lbl_pck.pickle','rb') as labels:
    lbl_arr=pickle.load(labels)

In [4]:
# Printing out the image array size and shape:
print("Number of Images = ",len(img_arr))
print("Shape of the Image Array = ",img_arr.shape)

Number of Images =  364305
Shape of the Image Array =  (364305, 50, 50, 1)


In [5]:
# Printing out the label array size and shape:
print("Number of Labels = ",len(lbl_arr))
print("Shape of the Label Array = ",lbl_arr.shape)

Number of Labels =  364305
Shape of the Label Array =  (364305,)


In [6]:
# Reshaping the label array:
lbl_arr = lbl_arr.reshape(364305,1)

In [7]:
# Checking the reshaped array:
print("Shape of the Label Array = ",lbl_arr.shape)

Shape of the Label Array =  (364305, 1)


In [9]:
# Loading all the NN related dependencies:

from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.models import load_model

In [11]:
cnn_model = Sequential(
    [
        # Convolutional Layer 1:
        Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=images.shape[1:]),
        
        # Spooling 1:
        MaxPooling2D(pool_size=(1, 1)),
        
        # Convolutional Layer 2:
        Conv2D(32, (3, 3), activation="relu"),
        
        # Spooling 2:
        MaxPooling2D(pool_size=(1, 1)),
        
        # Convolutional Layer 3:
        Conv2D(64, (3, 3), activation="relu"),
        
        # Spooling 3:
        MaxPooling2D(pool_size=(1, 1)),
        
        # Convolutional Droput Layer with 25% Dropout Rate:
        Dropout(0.25),
        
        # Flattening:
        Flatten(),
        
        # Dense Layer:
        Dense(128, activation="relu"),
        
        # Dense Dropout Layer with 50% Dropout Rate:
        Dropout(0.5),
        
        # Output layer with Softmax Output Function:
        Dense(2, activation="softmax"),
    ]
)

# Compiling the model with Adam optimizer and using the Accuracy Metric:
cnn_model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [12]:
# Displaying the model architecture:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 46, 46, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 44, 44, 64)       0

In [13]:
# Checking if GPUs are available in the training environment. We will be looking at training on a single GPU as training on multiple GPUs require Distribution Strategies:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

tf.config.list_physical_devices(device_type=None)

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

* We can use a trained model without having to retrain it, or pick-up training where we left off in case the training process was interrupted
* The `tf.keras.callbacks.ModelCheckpoint` callback will allow us to continually save the model both during and at the end of training

In [15]:
# Creating the checkpoint directory:
checkpoint_path = "/kaggle/working/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [16]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

In [ ]:
# Training the model:
with tf.device('/GPU:0'):
    cnn_model.fit(images, labels, batch_size=32, epochs=20, validation_split=0.02)

Epoch 1/50
5000/5000 [==============================] - ETA: 0s - loss: 0.4442 - accuracy: 0.8187

In [ ]:
# Saving the trained model:
model.save("/kaggle/working/ddd_CNN_model_v1")

In [ ]:
# To load the model back, execute this cell:
model = models.load_model("/kaggle/working/ddd_CNN_model")